In [1]:
from network import NeuralNetwork, Dense, linear, relu
import gymnasium as gym
import numpy as np

ImportError: cannot import name 'NeuralNetwork' from 'network' (/opt/homebrew/Caskroom/miniconda/base/envs/rl_exercises/lib/python3.10/site-packages/network.py)

In [ ]:
!pip install network

  Using cached network-0.1-py3-none-any.whl


In [ ]:
env = gym.make('CartPole-v1')
num_actions = env.action_space.n
obs_shape = env.observation_space.shape[0]

# Das Q-Net erhält als Eingabe einen State und berechnet für alle möglichen
# Aktionen einen Q-Wert.
q_net = NeuralNetwork([
    Dense(obs_shape, 32, activation=relu),
    Dense(32, 32, activation=relu),
    Dense(32, num_actions, activation=linear),
])

# Hier beispielhaft an der Transition zwischen initialen State und dem darauf
# folgenden State. Üblicherweise wird hier ein Batch aus dem ReplayBuffer
# verwendet.
state, info = env.reset()
action = env.action_space.sample()
next_state, reward, terminated, truncated, _ = env.step(action)
done = terminated or truncated

# Berechne Q-Values für alle möglichen Aktionen
q_values = q_net.forward(state)
print(q_values)

[[-0.18440509  0.12435395]]


## Bellman-Gleichung und Temporal Difference Error
Bellman-Gleichung: $Q^\pi(s, a) = r + \gamma Q^\pi(s', \pi(s'))$ <br><br>
Die Policy $\pi(s) = \arg\max_a Q(s, a)$ berechnet für einen gegebenen Zustand die Aktion mit dem höchsten Q-Wert. <br>
Folglich ist $Q^\pi(s', \pi(s'))$ der Q-Wert genau dieser Aktion bei gegebenen State.

Für das Training des neuronalen Netzes ergibt sich hier in der Praxis aber ein Problem. Der _Temporal Difference Errror_ berechnet sich aus dem Q-Wert der ausgeführten Aktion $a$ in State $s$ und der Aktion mit dem höchsten Q-Wert $\pi(s')$ im Zustand $s'$.
Das neuronale Netz berechnet allerdings Q-Werte für alle Aktionen gleichzeitig (aus Effizienzgründen). <br><br>

In [ ]:
# Berechne Bellman-Gleichung
GAMMA = 0.95

# Rechte Seite des Temporal Difference Errors
q_target = reward + GAMMA * np.max(q_net.forward(next_state), axis=1) * (1 - done)

print(q_target)

[1.95433478]


Wir müssen den berechneten Fehler (_Temporal Difference Error_) also so anpassen, dass wir damit unser Netz trainieren können. <br>

Temporal Difference Error: $\delta = Q^\pi(s, a) = r + \gamma Q^\pi(s', \pi(s'))$

In [ ]:
# Beispielhafte Lösung.
q_target = q_net.forward(state)
q_target[0, action] = reward + GAMMA * np.max(q_net.forward(next_state), axis=1)

print(q_target)

[[1.95433478 0.12435395]]


In [ ]:
q_values - q_target

array([[-2.13873986,  0.        ]])

Außerdem ist es für das Training erforderlich, dass wir eine Fehlerfunktion verwenden, bei der wir einem Gradienten folgen können. Hier können wir z.&nbsp;B. den berechnten Temporal Difference Error einfach quadrieren (MSE).

In [ ]:
(q_values - q_target)**2

array([[4.57420821, 0.        ]])